In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
bot_names = ["stale", "codecov", "jupyterlab-dev-mode", "henchbot"]
comments = pd.read_csv('./data/comments.csv', index_col=0).query('author not in @bot_names')
issues = pd.read_csv('./data/issues.csv', index_col=0).query('author not in @bot_names')
prs = pd.read_csv('./data/prs.csv', index_col=0).query('author not in @bot_names')

In [4]:
fmt_date = "{:%Y-%m-%d}"

start_date = comments['updatedAt'].min()
end_date = comments['updatedAt'].max()

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]
bot_names = ["stale", "codecov", "jupyterlab-dev-mode", "henchbot"]

In [5]:
# Parameters
end_date = "2019-07-23"
start_date = "2019-06-23"
renderer = "kaggle"
update_data = True


In [6]:
alt.renderers.enable(renderer);

In [7]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [8]:
# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [9]:
orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/nteract](https://github.com/nteract)
* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/ipython](https://github.com/ipython)
* [github.com/binder-examples](https://github.com/binder-examples)

In [10]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-06-23** to **2019-07-23**

## List of all contributors per organization

First, we'll list each contributor that has contributed to each organization in the last several days.
Contributions to open source projects are diverse, and involve much more than just contributing code and
code review. Thanks to everybody in the Jupyter communities for all that they do.

In [11]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_comments = comments.query('org == @org')['author']
    authors_prs = prs.query('org == @org')['author']
    unique_participants = np.unique(np.hstack([authors_comments.values, authors_prs.values]).astype(str)).tolist()
    unique_participants.sort(key=lambda a: a.lower())
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"All participants across issues and pull requests in each org in the last {n_days} days"))
display(tabs)

All participants across issues and pull requests in each org in the last 30 days

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [12]:
merged = prs.query('state == "MERGED"')

In [13]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                  org                         repo  author
  133          nteract                      nteract     484
  127          nteract                     hydrogen      84
  121          nteract                    bookstore      72
  135          nteract                    papermill      59
  134          nteract                   nteract.io      54
  136          nteract                    scrapbook      21
  132          nteract                          nes      18
  139          nteract                         vdom      11
  130          nteract              meeting-minutes       9
  137          nteract                     semiotic       9
  124          nteract                         docs       8
  131          nteract               minimal-plotly       8
  122          nteract                     commuter       6
  125          nteract                     examples       4
  138          nteract                semiotic-docs       4
  140          nteract                vega-embed-v3       2
  123          nteract           create-nteract-app       1
  126          nteract   graphql-schema-exploration       1
  128          nteract                          ion       1
  129          nteract                 kernel-relay       1
  98        jupyterlab                   jupyterlab     581
  103       jupyterlab               jupyterlab-git      62
  97        jupyterlab            jupyter-renderers      21
  113       jupyterlab               jupyterlab-toc      18
  100       jupyterlab        jupyterlab-commenting      15
  101       jupyterlab     jupyterlab-data-explorer      14
  99        jupyterlab          jupyterlab-celltags      13
  111       jupyterlab         jupyterlab-statusbar      13
  115       jupyterlab            jupyterlab_server      13
  107       jupyterlab  jupyterlab-metadata-service      12
  ..               ...                          ...     ...
  52           jupyter                    terminado       2
  23           jupyter                accessibility       1
  24           jupyter   cookiecutter-docker-stacks       1
  29           jupyter                         help       1
  41           jupyter           nbconvert-examples       1
  49           jupyter     sphinxcontrib_github_alt       1
  50           jupyter                      surveys       1
  19           ipython                      ipython     208
  17           ipython                    ipykernel      22
  22           ipython                    traitlets      10
  18           ipython                  ipyparallel       8
  20           ipython             ipython-in-depth       7
  21           ipython              ipython-website       4
  2    binder-examples             continuous-build       4
  0    binder-examples                  apt_install       3
  3    binder-examples                   data-quilt       2
  8    binder-examples                        latex       2
  10   binder-examples          multi-language-demo       2
  1    binder-examples                        conda       1
  4    binder-examples                   demo-julia       1
  5    binder-examples           getting-data-xroot       1
  6    binder-examples                  jupyter-sos       1
  7    binder-examples               jupyter-stacks       1
  9    binder-examples           minimal-dockerfile       1
  11   binder-examples                          nix       1
  12   binder-examples             python-conda_pip       1
  13   binder-examples                r_with_python       1
  14   binder-examples                 requirements       1
  15   binder-examples                     setup.py       1
  16   binder-examples                   stencila-r       1
  
  [141 rows x 3 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['nteract', 'hydrogen', 'bookstore', 'papermill', 'nteract.io', 'scrapbook', 'nes', 'vdom', 'm

### A list of merged PRs by project

Below is a tabbed readout of recently-merged PRs. Check out the title to get an idea for what they
implemented, and be sure to thank the PR author for their hard work!

In [14]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['author']
            user_url = author_url(user_name)
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['mergedBy']
            pr_closedby_url = f"https://github.com/{pr_closedby}"
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

### Authoring and merging stats by repository

Let's see who has been doing most of the PR authoring and merging. The PR author is generally the
person that implemented a change in the repository (code, documentation, etc). The PR merger is
the person that "pressed the green button" and got the change into the main codebase.

In [15]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'author'], as_index=False).agg({'id': 'count', 'authorAssociation': 'first'}).rename(columns={'id': "authored", 'author': 'username'})
closed_by_repo = merged.groupby(['org', 'repo', 'mergedBy']).count()['id'].reset_index().rename(columns={'id': "closed", "mergedBy": "username"})

In [16]:
charts = []
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)

alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                 org                 repo         username  authored  \
    0   binder-examples          apt_install           kannes         3   
    1   binder-examples                conda        emigre459         1   
    2   binder-examples     continuous-build          betatim         2   
    3   binder-examples     continuous-build          davidar         2   
    4   binder-examples           data-quilt      matheusmota         2   
    5   binder-examples           demo-julia     davidanthoff         1   
    6   binder-examples   getting-data-xroot  matthewfeickert         1   
    7   binder-examples          jupyter-sos           BoPeng         1   
    8   binder-examples       jupyter-stacks          parente         1   
    9   binder-examples                latex        bsyouness         1   
    10  binder-examples                latex        yuvipanda         1   
    11  binder-examples   minimal-dockerfile          betatim         1   
    12  binder-examples  multi-language-demo        choldgraf         1   
    13  binder-examples  multi-language-demo       jamesmyatt         1   
    14  binder-examples                  nix          betatim         1   
    15  binder-examples     python-conda_pip        fomightez         1   
    16  binder-examples        r_with_python          sebkopf         1   
    17  binder-examples         requirements          betatim         1   
    18  binder-examples             setup.py          cranmer         1   
    19  binder-examples           stencila-r       benmarwick         1   
    
       authorAssociation  
    0        CONTRIBUTOR  
    1        CONTRIBUTOR  
    2             MEMBER  
    3        CONTRIBUTOR  
    4               NONE  
    5        CONTRIBUTOR  
    6        CONTRIBUTOR  
    7       COLLABORATOR  
    8        CONTRIBUTOR  
    9        CONTRIBUTOR  
    10            MEMBER  
    11            MEMBER  
    12            MEMBER  
    13       CONTRIBUTOR  
    14            MEMBER  
    15       CONTRIBUTOR  
    16            MEMBER  
    17            MEMBER  
    18       CONTRIBUTOR  
    19       CONTRIBUTOR  ,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:          org              repo          username  authored authorAssociation
    20   ipython         ipykernel     AhmedRedaAmin         1       CONTRIBUTOR
    21   ipython         ipykernel           Carreau         4            MEMBER
    22   ipython         ipykernel           Chilipp         1       CONTRIBUTOR
    23   ipython         ipykernel         blink1073         1            MEMBER
    24   ipython         ipykernel           hroncok         1       CONTRIBUTOR
    25   ipython         ipykernel      jenshnielsen         1       CONTRIBUTOR
    26   ipython         ipykernel            maapxa         1       CONTRIBUTOR
    27   ipython         ipykernel   meeseeksmachine         1              NONE
    28   ipython         ipykernel             minrk         6            MEMBER
    29   ipython         ipykernel              n8vm         1       CONTRIBUTOR
    30   ipython         ipykernel         rayosborn         1       CONTRIBUTOR
    31   ipython         ipykernel          steff456 

In [17]:
charts = []
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                 org                 repo   username  closed
    0   binder-examples          apt_install    betatim       2
    1   binder-examples          apt_install  yuvipanda       1
    2   binder-examples                conda  choldgraf       1
    3   binder-examples     continuous-build    betatim       2
    4   binder-examples     continuous-build      minrk       1
    5   binder-examples     continuous-build  yuvipanda       1
    6   binder-examples           data-quilt    betatim       1
    7   binder-examples           data-quilt  choldgraf       1
    8   binder-examples           demo-julia    betatim       1
    9   binder-examples   getting-data-xroot  choldgraf       1
    10  binder-examples          jupyter-sos  choldgraf       1
    11  binder-examples       jupyter-stacks  yuvipanda       1
    12  binder-examples                latex  choldgraf       2
    13  binder-examples   minimal-dockerfile      minrk       1
    14  binder-examples  multi-language-demo  choldgraf       2
    15  binder-examples                  nix    betatim       1
    16  binder-examples     python-conda_pip  choldgraf       1
    17  binder-examples        r_with_python    betatim       1
    18  binder-examples         requirements    betatim       1
    19  binder-examples             setup.py    betatim       1
    20  binder-examples           stencila-r      nuest       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:         org              repo     username  closed
    21  ipython         ipykernel      Carreau       5
    22  ipython         ipykernel    blink1073       3
    23  ipython         ipykernel        minrk      14
    24  ipython       ipyparallel        minrk       8
    25  ipython           ipython      Carreau     185
    26  ipython           ipython       ivanov       1
    27  ipython           ipython  meeseeksdev       3
    28  ipython           ipython        minrk      16
    29  ipython           ipython       rgbkrk       1
    30  ipython           ipython    takluyver       2
    31  ipython  ipython-in-depth      Carreau       2
    32  ipython  ipython-in-depth    blink1073       3
    33  ipython  ipython-in-depth        minrk       1
    34  ipython  ipython-in-depth       rgbkrk       1
    35  ipython   ipython-website      Carreau       3
    36  ipython   ipython-website    blink1073       1
    37  ipython         traitlets        minrk       5
    38  ipython         traitlets     rmorshea       5,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for ipython in the last 30 days',
    width: 1000
  }), Chart({
    data:          org                        repo       username  closed
    39   jupyter               accessibility      ellisonbg       1
    40   jupyter  cookiecutter-docker-stacks        parente       1
    41   jupyter               docker-stacks          minrk       9
    42   jupyter               docker-stacks        parente      59
    43   jupyter       enhancement-proposals        Zsailer       1
    44   jupyter       enhancement-proposals   captainsafia       1
    45   jupyter       enhancement-proposals        parente       2
    46   jupyter       enhancement-proposals       willingc       3
    47   jupyter          enterprise_gateway      akchinSTC       3
    48   jupyter          enterprise_gateway    kevin-bates      47
    49   jupyter          enterprise_gateway       lresende      46
    50   jupyter                  governance  SylvainCorlay       1
    51   jupyter    

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [18]:
created = issues.query('state == "OPEN" and createdAt > @start_date and createdAt < @end_date')
closed = issues.query('state == "CLOSED" and closedAt > @start_date and closedAt < @end_date')

In [19]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                         repo  number  \
  0           ipython                    ipykernel       1   
  1           ipython                  ipyparallel       1   
  2           ipython                      ipython       6   
  3           ipython             ipython-in-depth       1   
  4           jupyter                accessibility       1   
  5           jupyter                docker-stacks       6   
  6           jupyter           enterprise_gateway       3   
  7           jupyter                      jupyter       4   
  8           jupyter                 jupyter-book       9   
  9           jupyter                jupyter-drive       1   
  10          jupyter               jupyter-sphinx       7   
  11          jupyter               jupyter_client       2   
  12          jupyter                 jupyter_core       1   
  13          jupyter               jupyter_server       2   
  14          jupyter                        nb2kg       1   
  15          jupyter                    nbconvert       5   
  16          jupyter                       nbdime       3   
  17          jupyter                     nbgrader       5   
  18          jupyter                     notebook      26   
  19          jupyter                    qtconsole       1   
  20          jupyter                  repo2docker      12   
  21          jupyter                    telemetry       1   
  22          jupyter                     testpath       1   
  23  jupyter-widgets                   ipyleaflet       4   
  24  jupyter-widgets                   ipywidgets      28   
  25  jupyter-widgets           jupyterlab-sidecar       3   
  26       jupyterhub                 batchspawner       3   
  27       jupyterhub                    binderhub      10   
  28       jupyterhub                dockerspawner       4   
  29       jupyterhub                   helm-chart       1   
  ..              ...                          ...     ...   
  31       jupyterhub         jupyter-server-proxy       2   
  32       jupyterhub                   jupyterhub      19   
  33       jupyterhub        kerberosauthenticator       1   
  34       jupyterhub                  kubespawner       2   
  35       jupyterhub            ldapauthenticator       4   
  36       jupyterhub          mybinder.org-deploy       4   
  37       jupyterhub          nativeauthenticator       1   
  38       jupyterhub               systemdspawner       3   
  39       jupyterhub                 team-compass       6   
  40       jupyterhub      the-littlest-jupyterhub      11   
  41       jupyterhub                traefik-proxy       2   
  42       jupyterhub                  wrapspawner       1   
  43       jupyterhub       zero-to-jupyterhub-k8s      11   
  44       jupyterlab            jupyter-renderers       2   
  45       jupyterlab                   jupyterlab      68   
  46       jupyterlab          jupyterlab-celltags       3   
  47       jupyterlab        jupyterlab-commenting      12   
  48       jupyterlab     jupyterlab-data-explorer      11   
  49       jupyterlab               jupyterlab-git       6   
  50       jupyterlab  jupyterlab-metadata-service       1   
  51       jupyterlab               jupyterlab-toc       1   
  52       jupyterlab              jupyterlab_apod       1   
  53       jupyterlab               jupyterlab_app       1   
  54       jupyterlab                 team-compass       3   
  55          nteract                    bookstore       3   
  56          nteract                  coffee_boat       1   
  57          nteract                     hydrogen       8   
  58          nteract                      nteract       4   
  59          nteract                    papermill       7   
  60          nteract                     semiotic       4   
  
                                    org/repo  
  0                        ipython/ipykernel  
  1                      ipython/ipyparallel 

In [20]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                       repo  number  \
  0   binder-examples                 demo-julia       1   
  1           ipython                  ipykernel       1   
  2           ipython                    ipython      16   
  3           ipython                  traitlets       2   
  4           jupyter              docker-stacks       2   
  5           jupyter         enterprise_gateway       6   
  6           jupyter               jupyter-book       6   
  7           jupyter             jupyter-sphinx      10   
  8           jupyter          jupyter.github.io       1   
  9           jupyter             jupyter_client       6   
  10          jupyter             jupyter_server       1   
  11          jupyter                      nb2kg       2   
  12          jupyter                  nbconvert       2   
  13          jupyter                     nbdime       1   
  14          jupyter                   nbgrader       1   
  15          jupyter                   nbviewer       2   
  16          jupyter                   notebook      14   
  17          jupyter                  qtconsole       2   
  18          jupyter                repo2docker       5   
  19          jupyter                  telemetry       3   
  20  jupyter-widgets                 ipyleaflet      10   
  21  jupyter-widgets                 ipywidgets      20   
  22  jupyter-widgets         jupyterlab-sidecar       1   
  23  jupyter-widgets               midicontrols       1   
  24  jupyter-widgets                  pythreejs       1   
  25  jupyter-widgets                   tutorial       1   
  26       jupyterhub               batchspawner       1   
  27       jupyterhub                  binderhub       5   
  28       jupyterhub              dockerspawner       3   
  29       jupyterhub       jupyter-server-proxy       1   
  30       jupyterhub                 jupyterhub       5   
  31       jupyterhub   jupyterhub-deploy-docker       1   
  32       jupyterhub      kerberosauthenticator       1   
  33       jupyterhub                kubespawner       3   
  34       jupyterhub        mybinder.org-deploy       4   
  35       jupyterhub             oauthenticator       1   
  36       jupyterhub               team-compass      10   
  37       jupyterhub    the-littlest-jupyterhub       5   
  38       jupyterhub              traefik-proxy       4   
  39       jupyterhub     zero-to-jupyterhub-k8s       8   
  40       jupyterlab  extension-cookiecutter-js       1   
  41       jupyterlab          jupyter-renderers       3   
  42       jupyterlab                 jupyterlab      53   
  43       jupyterlab   jupyterlab-data-explorer       3   
  44       jupyterlab             jupyterlab-git       4   
  45       jupyterlab             jupyterlab-toc       2   
  46          nteract                  bookstore       5   
  47          nteract                   commuter       1   
  48          nteract                   hydrogen      29   
  49          nteract            meeting-minutes       1   
  50          nteract                    nteract      14   
  51          nteract                  papermill       6   
  52          nteract                  scrapbook       1   
  53          nteract                   semiotic       4   
  54          nteract                       vdom       1   
  
                                  org/repo  
  0             binder-examples/demo-julia  
  1                      ipython/ipykernel  
  2                        ipython/ipython  
  3                      ipython/traitlets  
  4                  jupyter/docker-stacks  
  5             jupyter/enterprise_gateway  
  6                   jupyter/jupyter-book  
  7                 jupyter/jupyter-sphinx  
  8              jupyter/jupyter.github.io  
  9                 jupyter/jupyter_client  
  10                jupyter/jupyter_server  
  11                         jupyter/nb2kg  
  12                     jupyter/nbcon

### A list of recent issues

Below is a list of issues with recent activity in each repository. If they seem of interest
to you, click on their links and jump in to participate!

In [21]:
# Add comment count data to issues and PRs
comment_counts = comments.groupby(['org', 'repo', 'issue_id']).count().iloc[:, 0].to_frame()
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [22]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(comment_counts.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for (org, repo), i_issues in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        
        top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
        top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'number'], right_on=['org', 'repo', 'issue_id'])
        for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['author']
            user_url = author_url(user_name)
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)
    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 30 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [23]:
commentors = (
    comments
    .groupby(['org', 'repo', 'author', 'authorAssociation'])
    .count().rename(columns={'id': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [24]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='count',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                 org                 repo               author  \
    25  binder-examples         getting-data              betatim   
    36  binder-examples           jupyterlab            choldgraf   
    54  binder-examples                    r            choldgraf   
    21  binder-examples           demo-julia            choldgraf   
    27  binder-examples         getting-data      matthewfeickert   
    28  binder-examples         getting-data             stwunsch   
    2   binder-examples          apt_install               kannes   
    15  binder-examples     continuous-build                vsoch   
    10  binder-examples     continuous-build              betatim   
    52  binder-examples                    r              betatim   
    34  binder-examples          jupyter-sos            choldgraf   
    37  binder-examples           jupyterlab          cornhundred   
    63  binder-examples        r_with_python              betatim   
    18  binder-examples           data-quilt          matheusmota   
    53  binder-examples                    r             cboettig   
    59  binder-examples                    r               mrjoh3   
    3   binder-examples          apt_install            yuvipanda   
    22  binder-examples           demo-julia         davidanthoff   
    24  binder-examples         getting-data               akarve   
    33  binder-examples          jupyter-sos               BoPeng   
    58  binder-examples                    r      gedankenstuecke   
    62  binder-examples                    r                vsoch   
    64  binder-examples        r_with_python              marskar   
    0   binder-examples          apt_install              betatim   
    4   binder-examples                bokeh            choldgraf   
    6   binder-examples                conda  3keepmovingforward3   
    13  binder-examples     continuous-build              jhamman   
    17  binder-examples           data-quilt            choldgraf   
    20  binder-examples           demo-julia              betatim   
    35  binder-examples           jupyterlab        YannickJadoul   
    39  binder-examples                latex            choldgraf   
    42  binder-examples  multi-language-demo            choldgraf   
    43  binder-examples  multi-language-demo           jamesmyatt   
    50  binder-examples                    r         RaoOfPhysics   
    55  binder-examples                    r             coatless   
    61  binder-examples                    r         taylorreiter   
    65  binder-examples        r_with_python              sebkopf   
    66  binder-examples         requirements            choldgraf   
    68  binder-examples             setup.py            choldgraf   
    1   binder-examples          apt_install            choldgraf   
    5   binder-examples                bokeh  mathematicalmichael   
    7   binder-examples                conda              betatim   
    8   binder-examples                conda            choldgraf   
    9   binder-examples                conda          robmarkcole   
    11  binder-examples     continuous-build            choldgraf   
    12  binder-examples     continuous-build            fomightez   
    14  binder-examples     continuous-build      matthewfeickert   
    16  binder-examples           data-quilt              betatim   
    19  binder-examples           demo-julia            Krastanov   
    23  binder-examples           demo-julia              jzf2101   
    
       authorAssociation  count  
    25            MEMBER     11  
    36            MEMBER     11  
    54            MEMBER     11  
    21            MEMBER      9  
    27              NONE      8  
    28              NONE      8  
    2        CONTRIBUTOR      7  
    15       CONTRIBUTOR      7  
    10            MEMBER      6  
    52            MEMBER      6  
    34            MEMBER      5  
    37              NONE   

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [25]:
first_comments = []
for (org, repo, issue_id), i_comments in comments.groupby(['org', 'repo', 'issue_id']):
    ix_min = pd.to_datetime(i_comments['createdAt']).idxmin()
    first_comments.append(i_comments.loc[ix_min])
first_comments = pd.concat(first_comments, axis=1).T

In [26]:
first_responder_counts = first_comments.groupby(['org', 'author', 'authorAssociation'], as_index=False).\
    count().rename(columns={'id': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)

In [27]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(first_responder_counts.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='n_first_responses',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                 org               author authorAssociation  createdAt  \
    4   binder-examples            choldgraf            MEMBER         24   
    3   binder-examples              betatim            MEMBER         11   
    9   binder-examples               kannes       CONTRIBUTOR          3   
    2   binder-examples         RaoOfPhysics       CONTRIBUTOR          2   
    0   binder-examples  3keepmovingforward3              NONE          1   
    1   binder-examples            Krastanov              NONE          1   
    5   binder-examples              cranmer       CONTRIBUTOR          1   
    6   binder-examples      gedankenstuecke            MEMBER          1   
    7   binder-examples           jamesmyatt       CONTRIBUTOR          1   
    8   binder-examples              jhamman              NONE          1   
    10  binder-examples      matthewfeickert       CONTRIBUTOR          1   
    11  binder-examples      matthewfeickert              NONE          1   
    12  binder-examples                minrk            MEMBER          1   
    13  binder-examples               mrjoh3              NONE          1   
    14  binder-examples                nuest            MEMBER          1   
    15  binder-examples              sebkopf            MEMBER          1   
    16  binder-examples         taylorreiter              NONE          1   
    17  binder-examples                vsoch       CONTRIBUTOR          1   
    18  binder-examples            yuvipanda            MEMBER          1   
    
        updatedAt  url  repo  issue_id  n_first_responses  
    4          24   24    24        24                 24  
    3          11   11    11        11                 11  
    9           3    3     3         3                  3  
    2           2    2     2         2                  2  
    0           1    1     1         1                  1  
    1           1    1     1         1                  1  
    5           1    1     1         1                  1  
    6           1    1     1         1                  1  
    7           1    1     1         1                  1  
    8           1    1     1         1                  1  
    10          1    1     1         1                  1  
    11          1    1     1         1                  1  
    12          1    1     1         1                  1  
    13          1    1     1         1                  1  
    14          1    1     1         1                  1  
    15          1    1     1         1                  1  
    16          1    1     1         1                  1  
    17          1    1     1         1                  1  
    18          1    1     1         1                  1  ,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'n_first_responses',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 first responders for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:          org           author authorAssociation  createdAt  updatedAt  url  \
    24   ipython          Carreau            MEMBER        286        286  286   
    228  ipython            minrk            MEMBER         31         31   31   
    262  ipython         rmorshea      COLLABORATOR         27         27   27   
    37   ipython   LucianaMarques       CONTRIBUTOR         14         14   14 

In [28]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>